In [1]:
import os
os.chdir("/Users/eric/Projects/sentiment/")

In [2]:
import numpy as np

In [3]:
from app import db, app
from app.emojis import emojis
from app.models import Tweet

/Users/eric/Projects/sentiment/venv/lib/python2.7/site-packages/flask_sqlalchemy/__init__.py:800: UserWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.
  warnings.warn('SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True to suppress this warning.')
Using Theano backend.


In [4]:
from app.ml import *

In [5]:
gen = data_gen()

In [10]:
sm = SentimentModel()

In [11]:
sm._model.output_shape

[(None, 98), (None, 3)]

In [12]:
x, y= gen.next()

In [13]:
y[1].shape

(100, 3)

In [14]:
sm._model.loss_functions

[<function keras.objectives.binary_crossentropy>,
 <function keras.objectives.binary_crossentropy>]